In [1]:
pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 304.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 30.6 MB/s eta 0:00:00

In [2]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.5 MB/s eta 0:00:00


In [3]:
pip install openai langchain

In [15]:
import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

In [16]:
OPENAI_API_KEY = getpass()
llm = OpenAI(api_key= OPENAI_API_KEY)

··········


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [20]:


template = """This is a conversation between a human and ai agent:
{chat_history}
Write a summary of the conversation for {input}:
"""
prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)

search = DuckDuckGoSearchRun()

tool_use = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="Useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )
]

tool_use_1 = [
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="Useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )
]

email_author = Agent(
    role='Professional Email Author',
    goal='Craft concise and engaging emails',
    backstory='Experienced in writing impactful marketing emails.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use
)
marketing_strategist = Agent(
    role='Marketing Strategist',
    goal='Lead the team in creating effective cold emails',
    backstory='A seasoned Chief Marketing Officer with a keen eye for standout marketing content.',
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tool_use_1
)

content_specialist = Agent(
    role='Content Specialist',
    goal='Critique and refine email content',
    backstory='A professional copywriter with a wealth of experience in persuasive writing.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use_1
)

email_task = Task(
    description='''1. Generate two distinct variations of a cold email promoting a video editing solution.
    2. Evaluate the written emails for their effectiveness and engagement.
    3. Scrutinize the emails for grammatical correctness and clarity.
    4. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist.
    5. Revise the emails based on all feedback, creating two final versions.''',
    agent=marketing_strategist,  # O estrategista de marketing será o responsável por administrar esta tarefa
    expected_output="Two final versions of the cold email that meet the specified criteria."
)

In [27]:

email_crew = Crew(
    agents=[email_author, marketing_strategist, content_specialist],
    tasks=[email_task],
    verbose=True,
    process=Process.sequential
)


print("Crew: Working on Email Task")
emails_output = email_crew.kickoff()

Crew: Working on Email Task
 [2024-08-27 19:59:00][DEBUG]: == Working Agent: Marketing Strategist
 [2024-08-27 19:59:00][INFO]: == Starting Task: 1. Generate two distinct variations of a cold email promoting a video editing solution.
    2. Evaluate the written emails for their effectiveness and engagement.
    3. Scrutinize the emails for grammatical correctness and clarity.
    4. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist.
    5. Revise the emails based on all feedback, creating two final versions.


> Entering new CrewAgentExecutor chain...

I need to create two effective cold emails to promote a video editing solution. I must carefully evaluate and scrutinize the emails for engagement and correctness, and make necessary adjustments based on feedback and best practices. 

Action: Summary
Action Input: {"tool_input": "Marketing Strategist"}

> Entering new LLMChain chain...
Prompt after formatting: